In [51]:
#we load the libraries 
library(readr)
library(dplyr)
library(ggplot2)
library(tidyr)
library(quanteda)
library(lubridate)
library(stringr)


In [52]:
#we read the dataframe created before with all the articles, after they have been translated

library(readr)

articles <- read_csv('cleaned_articles_other_subjects.csv')
head(articles)

Rows: 920 Columns: 5
── Column specification ──────────────────────────────────────────────────
Delimiter: ","
chr  (3): article_url, article_text, cleaned_text
dttm (2): article_date_created, article_date_updated

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


article_url                                                                            
1 https://www.thejournal.ie/bujo-planning-permission-3-4544147-Mar2019/                  
2 https://www.the42.ie/tyrone-dublin-league-report-4546910-Mar2019/                      
3 https://www.thejournal.ie/missing-jamie-walsh-garda-appeal-rathfarnham-4556727-Mar2019/
4 https://www.thejournal.ie/dept-health-evacuated-4522908-Mar2019/                       
5 https://www.thejournal.ie/valentines-day-deals-supermarkets-4535989-Mar2019/           
6 https://www.thejournal.ie/brexit-newsletter-thejournal-4506615-Mar2019/                
  article_text                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

In [53]:
count(articles)

n
<int>
920


In [54]:
data("data_dictionary_LSD2015", package = "quanteda")


In [55]:
#before applying the dictionary we first need to tokenise the text:

articles_toks <- quanteda::tokens(
  articles$cleaned_text,
  remove_numbers = TRUE,
  remove_symbols = TRUE
)

In [56]:
#we apply Lexicoder Sentiment Dictionary to a document feature matrix
articles_sent <- quanteda::dfm(
  quanteda::tokens_lookup(articles_toks, dictionary = data_dictionary_LSD2015)
)
articles_sent

Document-feature matrix of: 920 documents, 4 features (45.33% sparse) and 0 docvars.
       features
docs    negative positive neg_positive neg_negative
  text1        8       21            1            0
  text2       27       27            0            0
  text3        0       14            0            0
  text4       11       24            0            0
  text5        6       25            0            0
  text6        2       14            0            0
[ reached max_ndoc ... 914 more documents ]

In [57]:
#we extract negative counts
negative_counts <- convert(articles_sent, to = "data.frame")$negative


In [65]:
#then we need to see the negativity percentage for each article, based on the article length
articles$article_length <- str_count(articles$article_text, "\\S+")
articles <- articles %>%
  mutate(negativity_percent = (negativity / article_length)*100)


In [71]:
count(articles)

n
<int>
920


## Now we can compare the sentiment of the Brexit articles and of the other articles ##

In [61]:
articles_brexit <- read_csv('final_df.csv')
head(articles_brexit)

Rows: 2532 Columns: 13
── Column specification ──────────────────────────────────────────────────
Delimiter: ","
chr  (6): article_url, article_text, cleaned_text, month, assigned_topic, au...
dbl  (5): num_comments, year, negativity, article_length, negativity_percent
date (2): article_date_created, article_date_updated

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


article_url                                                                                            
1 https://www.thejournal.ie/government-seeking-early-warning-system-on-use-of-article-16-5350126-Feb2021/
2 https://www.thejournal.ie/rishi-sunak-northern-ireland-protocol-deal-imminent-5997518-Feb2023/         
3 https://www.thejournal.ie/migration-uk-eu-after-brexit-5561000-Oct2021/                                
4 https://www.thejournal.ie/irish-exports-highest-level-ever-5683731-Feb2022/                            
5 https://www.thejournal.ie/uk-bill-visa-non-eu-citizens-northern-ireland-border-5625136-Dec2021/        
6 https://www.thejournal.ie/no-extension-brexit-residency-scheme-uk-5474654-Jun2021/                     
  article_text                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              

In [72]:
#we filter the prexit articles for the year 2019
brexit_2019 <- articles_brexit %>% filter(year == 2019)
count(brexit_2019)

n
<int>
905


In [77]:
mean(brexit_2019$negativity, na.rm = TRUE)
mean(articles$negativity, na.rm = TRUE)


[1] 21.12265

[1] 20.85652

In [78]:
#we perform a two-sided T-test
t.test(brexit_2019$negativity, articles$negativity)



	Welch Two Sample t-test

data:  brexit_2019$negativity and articles$negativity
t = 0.28906, df = 1612.3, p-value = 0.7726
alternative hypothesis: true difference in means is not equal to 0
95 percent confidence interval:
 -1.539710  2.071971
sample estimates:
mean of x mean of y 
 21.12265  20.85652 


In [ ]:
#The p-value is 0.7726, which is very high 
#this means there’s no statistically significant difference between the two averages
#The confidence interval (-1.54 to 2.07) includes zero, also indicating no meaningful difference

#Brexit stories in 2019 are just as negative as other stories in the Journal.ie
#There’s no real difference in negativity between them based on the scores

